In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
trash_pickup_df=pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. One category of liquidated damages is related to chronic problems in any category of service at the same premises. A chronic problem is defined as more than one missed pickup for any address. The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

In [3]:
trash_pickup_df.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [4]:
trash_pickup_df.tail()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016,-86.741242,36.169482
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637
20225,267137,11/1/2019,Trash - Curbside/Alley Missed Pickup,possibly others missed as well,"604 Somerset Ct, Nashville, TN 37217, United S...",37217.0,RED RIVER,2505,29.0,1781360.323,637742.0068,-86.633331,36.083675


In [5]:
trash_pickup_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


In [6]:
trash_pickup_df.shape

(20226, 13)

In [7]:
rr_trash=trash_pickup_df[trash_pickup_df['Trash Hauler'] == 'RED RIVER']

In [8]:
rr_trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140


In [9]:
rr_trash.shape

(14395, 13)

In [10]:
rr_missed_trash=rr_trash[rr_trash['Request'].str.contains('Missed Pickup', case=False, na=False)]

In [11]:
rr_missed_trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728
10,25341,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed trash pickup - resident has at curb for...,3113 HYDES FERRY RD,37218.0,RED RIVER,4204,2.0,1721577.768,676018.3999,-86.836750,36.187584


In [12]:
rr_missed_trash.shape

(10715, 13)

In [26]:
rr_group_trash=rr_missed_trash.groupby('Incident Address')['Request'].count().sort_index()

In [27]:
rr_group_trash.tail

<bound method NDFrame.tail of Incident Address
100 Bluefield Square, Nashville, TN 37214, United States    1
100 Braxton Hill Ct                                         1
100 Brook Hollow Rd                                         1
100 Clydelan Ct, Nashville, TN 37205, United States         2
100 Desoto Dr, Nashville, TN 37210, United States           1
                                                           ..
Old Tusculum Rd, Antioch, Tennessee, 37013                  1
Queens Ln, Nashville, TN 37218, United States               1
Robin Springs Rd                                            1
Trevino Pl, Nashville, TN 37013, United States              1
Westboro Dr, Nashville, TN 37209, United States             1
Name: Request, Length: 8190, dtype: int64>

In [28]:
rr_group_trash = rr_group_trash[rr_group_trash > 1]

In [29]:
print(rr_group_trash)

Incident Address
100 Clydelan Ct, Nashville, TN 37205, United States        2
100 Lincoln Ct, Nashville, TN 37205, United States         2
100 Nashboro Greens, Nashville, TN 37217, United States    2
100 Rhine Dr, Nashville, TN 37115, United States           2
1000 Flintlock Ct, Nashville, TN 37217, United States      2
                                                          ..
946 Youngs Ln                                              2
947 Glendale Ln, Nashville, TN 37204, United States        2
960 44th Ave N, Nashville, TN 37209, United States         2
971 Windrowe Dr, Nashville, TN 37205, United States        2
986 Malquin Dr, Nashville, TN 37216, United States         2
Name: Request, Length: 1551, dtype: int64


In [30]:
total_sum=rr_group_trash.sum()
print(total_sum)

4075


In [31]:
count_of_repeat=rr_group_trash.count()
print(count_of_repeat)

1551


4075 requests from addresses used more than once - 1551(1 for each address)= 2524 * 200 fine= $504800

* What other types of complaints are there?
* Are there any geospatial analysis you can do?  Which visualizations can you create?
* How do metro crews compare to the contractor's performance?
* How much does each trash hauler owe?
* What were to total missed pickup by route?

In [32]:
total_fines=(total_sum - count_of_repeat) * 200
print(total_fines)

504800


In [33]:
types_request=trash_pickup_df['Request'].unique()
print(types_request)

['Trash - Backdoor' 'Trash - Curbside/Alley Missed Pickup'
 'Trash Collection Complaint' 'Damage to Property']


In [34]:
missed_trash_all=trash_pickup_df[trash_pickup_df['Request'].str.contains('Missed Pickup', case=False, na=False)]

In [35]:
missed_trash_all.loc[:,'Trash Hauler'] = missed_trash_all['Trash Hauler'].str.upper()

In [36]:
grouped_missed_trash=missed_trash_all.groupby('Trash Hauler')['Request'].count().sort_index()

In [37]:
print(grouped_missed_trash)

Trash Hauler
METRO         2688
RED RIVER    10715
WASTE IND     1019
Name: Request, dtype: int64
